# Amino acid preferences
This notebook takes codon counts and uses dms_tools2 to calculate the amino acid preferences at mutagenezed sites in the Zika Virus NS5 protein.

## Notebook setup
Imports

In [1]:
import dms_tools2
import numpy as np
import os
import pandas as pd
import plotnine as p9
import regex
from Bio import SeqIO
from dms_tools2.ipython_utils import showPDF

# print imports and versions
# copied from https://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook
print('\n'.join(f'{m.__name__}=={m.__version__}' \
                for m in globals().values() if getattr(m, '__version__', None)))

dms_tools2==2.6.8
numpy==1.19.1
pandas==0.25.3
plotnine==0.6.0
regex==2.5.83


Input data

In [2]:
samplesheet = 'data/samplesheet.csv'
codon_counts_dir = 'results/codon_counts/'

Output data

In [3]:
results_dir = 'results/'

Analysis parameters

Notebook aesthetics

In [4]:
p9.theme_set(p9.theme_classic())
CBPALETTE_RICH = ['#648FFF', '#FFB000', '#DC267F', '#785EF0', '#FE6100']

## Load data
Load samples

In [7]:
samples = pd.read_csv(samplesheet)
samples = samples.drop('r1_file', axis=1)
samples.insert(0, 'name', (samples['library'] + '-' +
                           samples['source'] + '-' +
                           samples['stage'] + '-' +
                           samples['sorted']))
display(samples)

,name,library,source,stage,sorted
0,wt-plasmid-noStage-notSorted,wt,plasmid,noStage,notSorted
1,wt-gDNA-noStage-notSorted,wt,gDNA,noStage,notSorted
2,lib1-plasmid-noStage-notSorted,lib1,plasmid,noStage,notSorted
3,lib2-plasmid-noStage-notSorted,lib2,plasmid,noStage,notSorted
4,lib3-plasmid-noStage-notSorted,lib3,plasmid,noStage,notSorted
5,lib1-gDNA-early-notSorted,lib1,gDNA,early,notSorted
6,lib2-gDNA-early-notSorted,lib2,gDNA,early,notSorted
7,lib3-gDNA-early-notSorted,lib3,gDNA,early,notSorted
8,lib1-gDNA-mid-notSorted,lib1,gDNA,mid,notSorted
9,lib2-gDNA-mid-notSorted,lib2,gDNA,mid,notSorted


# Calculate amino acid preferences
Calculate the [amino acid preferences](https://jbloomlab.github.io/dms_tools2/prefs.html) of the sites on each mutagenized gDNA sample.

Setup:

In [14]:
# Set up results dir
prefs_dir = os.path.join(results_dir + 'prefs')
os.makedirs(prefs_dir, exist_ok=True)

# Make samplesheet
batchfile = os.path.join(prefs_dir + '/' + 'batchfile.csv')
# Code from Jesse to quickly automatically generate this.
batchfile_df = (
    samples
    .query('library != "wt"')
    .query('source != "plasmid"')
    .assign(post=lambda x: x['name'])
    .merge(samples.query('source == "plasmid"')
                  .assign(pre=lambda x: x['name'])
                  [['library', 'pre']],
           on=['library'], how='left', validate='many_to_one',
           )
    [['name', 'source', 'library', 'pre', 'post',]]
    .assign(errpre='wt-plasmid-noStage-notSorted')
    .merge(samples.query('library == "wt"')
                  .assign(errpost=lambda x: x['name'])
                  [['source', 'errpost']],
           on=['source'], how='left'
           )
    )
assert batchfile_df.notnull().all().all()
display(batchfile_df)

print(f'Writing batchfile to {batchfile}')
batchfile_df.to_csv(batchfile, index=False)
print('Done.')


,name,source,library,pre,post,errpre,errpost
0,lib1-gDNA-early-notSorted,gDNA,lib1,lib1-plasmid-noStage-notSorted,lib1-gDNA-early-notSorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted
1,lib2-gDNA-early-notSorted,gDNA,lib2,lib2-plasmid-noStage-notSorted,lib2-gDNA-early-notSorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted
2,lib3-gDNA-early-notSorted,gDNA,lib3,lib3-plasmid-noStage-notSorted,lib3-gDNA-early-notSorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted
3,lib1-gDNA-mid-notSorted,gDNA,lib1,lib1-plasmid-noStage-notSorted,lib1-gDNA-mid-notSorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted
4,lib2-gDNA-mid-notSorted,gDNA,lib2,lib2-plasmid-noStage-notSorted,lib2-gDNA-mid-notSorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted
5,lib3-gDNA-mid-notSorted,gDNA,lib3,lib3-plasmid-noStage-notSorted,lib3-gDNA-mid-notSorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted
6,lib1-gDNA-late-notSorted,gDNA,lib1,lib1-plasmid-noStage-notSorted,lib1-gDNA-late-notSorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted
7,lib2-gDNA-late-notSorted,gDNA,lib2,lib2-plasmid-noStage-notSorted,lib2-gDNA-late-notSorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted
8,lib3-gDNA-late-notSorted,gDNA,lib3,lib3-plasmid-noStage-notSorted,lib3-gDNA-late-notSorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted
9,lib1-gDNA-late-sorted,gDNA,lib1,lib1-plasmid-noStage-notSorted,lib1-gDNA-late-sorted,wt-plasmid-noStage-notSorted,wt-gDNA-noStage-notSorted


Writing batchfile to results/prefs/batchfile.csv
Done.


Command:

In [15]:
ncpus = 16

# Overwrite or use existing results
use_existing = 'no'

log = ! dms2_batch_prefs \
        --indir {codon_counts_dir} \
        --batchfile {batchfile} \
        --outdir {prefs_dir} \
        --summaryprefix summary \
        --use_existing {use_existing} \
        --ncpus {ncpus}

# assert all(map(os.path.isfile, [os.path.join(prefs_dir, name + '_prefs.csv') 
#                                 for name in batchfile_df.name])), '\n'.join(log)

print("Amino-acid preferences calculated for all samples.")

Amino-acid preferences calculated for all samples.


## Visualize results
**Reads and barcodes per sample:**

Interpretation:

**Barcode coverage:**

Interpretation:

**Mutation frequency:**

Interpretation:  

Intepretation: 